In [84]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 11
### Team Member Names: Akram, Annie, Jester
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


## **General Strategy for the Project**:

#### Initialization
0) Define and initialize necessary global variables

#### Part #1: Data Filtering and Cleaning
1) Filter out all valid US and CAD Stocks from the provided CSV file.
2) Download and store closing price, options and volume data in a dictionary using yfinance, where US stocks prices are converted using real-time exchange rate.
3) Filter out tickers within date range based on given minimum monthly average volume and minimum trading days in a month.

#### Part #2: Portfolio Construction
4) Rank stocks based on Standard Deviation of percentage change in returns in descending order.
5) Rank stocks based on PCR values using options data in descending order.
6) Score the stocks based on the two ranks, and create a new ranking based on the scoring.
7) We select stocks based on the ranking and calculate weights that would maximize the portfolio sharpe ratio while beta is within pre-defined constraints.
8) Run sharpe ratio calculation function on portfolio from size 12(min) to 24(max), so that the final portfolio (with weightings) is picked based on highest output sharpe ratio.

#### Part #3: Evaluation and Proof
9) Given the chosen portfolio of xx stocks, graph the change in portfolio standard deviation as other stocks are added.
10) Beta of portfolio compared to the S&P 500.
11) Calculate the Beta between our portfolio and an equally weighted portfolio including all valid stocks.
12) Beta between our portfolio with varied weight versus when the portfolio is equally weighted.
13) Sharpe ratio between our portfolio with varied weight versus when it is equally weighted.
14) Graphically compare sharpe ratios amongst our portfolio of xx stocks and the portfolios of varying 12-24 stocks.

#### Part #4: Final Output
15) Creating the final portfolio dataframe and CSV.

## Initializing Variables

In [85]:
def get_tickers():
    tickers = pd.read_csv('Tickers.csv')
    ticker_lst = [tickers.columns[0]] + (list(tickers[tickers.columns[0]]))
    return ticker_lst

In [86]:
# Important Constants: 
amount = 1_000_000 # Initial investment amount of $1,000,000
group = 11

# Define constants
min_avg_volume = 100000
min_trading_days = 18
start_date, end_date = '2022-09-30', '2024-09-30'
min_stocks, max_stocks = 12, 24

# Reading in CSV file: 
ticker_lst = get_tickers()

# Initializing variable to store the tickers we will use in our portfolio
columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
Portfolio_Final = pd.DataFrame(columns=columns)
exchange_rate = yf.Ticker('CAD=X').fast_info['last_price']
print(f'The current exchange rate for the latest available day:\nUSD -> CAD: ${np.round(exchange_rate, 4)}')

The current exchange rate for the latest available day:
USD -> CAD: $1.3973


#### We must filter the tickers csv as follows:
- Must be listed on yfinance
- The currency is listed as USD or CAD 
- 100,000+ average monthly volume trades
- More than 18 trades per month
- Sufficient data

In [87]:
# Filtering valid stocks by inputting a list of strings for each ticker. 
def filter_stocks(ticker_lst):
    # Function to drop short trading months (less than 18 trading days per month)
    def drop_short_trading_months(df):
        """
        Drops months with less than 18 trading days from a yfinance history DataFrame.
        Parameters:
            df (pd.DataFrame): A yfinance DataFrame with a DatetimeIndex and stock data.
        Returns:
            pd.DataFrame: Filtered DataFrame with only months having >= 18 trading days.
        """
        # Ensure the index is a DatetimeIndex
        if not isinstance(df.index, pd.DatetimeIndex):
            raise ValueError("The DataFrame index must be a DatetimeIndex.")
        # Remove timezone information to avoid warnings
        df = df.copy()  # Avoid modifying the original DataFrame
        df.index = df.index.tz_localize(None)
        # Group by year and month
        df['YearMonth'] = df.index.to_period('M')  # Creates a 'YearMonth' period
        # Count trading days for each month
        trading_days_per_month = df.groupby('YearMonth').size()
        # Get valid months with at least 18 trading days
        valid_months = trading_days_per_month[trading_days_per_month >= 18].index
        # Filter DataFrame to include only rows in valid months
        filtered_df = df[df['YearMonth'].isin(valid_months)].drop(columns=['YearMonth'])
        return filtered_df
    
    valid_tickers, invalid_tickers, usdstocks = {}, [], []
    # Loop through all tickers to check if they are valid
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        try:
            info = stock.fast_info # Get basic stock info

            hist = stock.history(start=start_date, end=end_date) # Get stock history
            pd.to_datetime(hist.index, format='%Y-%m-%d')
            
            avg_volume = hist.loc[((hist.index >= start_date) & (hist.index <= end_date))]['Volume'].mean() # Calculate average volume in specified date range.
            currency = info.get("currency")
            if ((hist.empty is not None) and # filter for stocks delisted on yfinance
                ( currency == "USD" or currency == "CAD") and # filter for stocks that are not USD
                (avg_volume >= min_avg_volume)): # Filter by volume greater than 100,000
                if currency == "CAD":
                    hist = drop_short_trading_months(hist)
                    hist.index = hist.index.strftime('%Y-%m-%d')
                    valid_tickers[ticker] = hist['Close'] # Store the close prices of the stock as a Series
                elif currency == "USD":
                    hist = drop_short_trading_months(hist)
                    hist.index = hist.index.strftime('%Y-%m-%d')
                    usdstocks.append(ticker)
                    valid_tickers[ticker] = hist['Close'] * exchange_rate # Convert USD to CAD
            else:
                invalid_tickers.append(ticker)
        except:
            invalid_tickers.append(ticker)
    return [valid_tickers, invalid_tickers, usdstocks]
    # valid_tickers is a dictionary of Series where the key is the name of the ticker. 
    # invalid_tickers is a list of ticker strings which were removed in the filtering process. 
    # usdstocks is a list of ticker strings which were converted from USD to CAD.

In [88]:
def calculate_std(data):
    data.index = pd.to_datetime(data.index)
    
    # Calculate daily percentage returns
    returns = data.pct_change(fill_method=None).dropna()

    # Calculate standard deviation of returns
    std = pd.DataFrame(returns.std(), columns=['Standard Deviation'])

    # Sort by standard deviation
    std_sorted = std.sort_values(by='Standard Deviation', ascending=False)

    # Add Rank column
    std_sorted['Rank'] = range(len(std_sorted))

    # Add Score column
    highest_std_value = std_sorted['Standard Deviation'].iloc[0]
    std_sorted['Score'] = (std_sorted['Standard Deviation'] / highest_std_value) * 100

    return std_sorted

In [89]:
def calculate_return(data):
    data.index = pd.to_datetime(data.index)
    
    # Calculate daily percentage returns
    returns = data.pct_change(fill_method=None).dropna()

    # Calculate standard deviation of returns
    ret = pd.DataFrame(returns.mean(), columns=['Return'])

    # Sort by standard deviation
    ret_sorted = ret.sort_values(by='Return', ascending=False)

    # Add Rank column
    ret_sorted['Rank'] = range(len(ret_sorted))

    # Add Score column
    highest_ret_value = ret_sorted['Return'].iloc[0]
    ret_sorted['Score'] = (ret_sorted['Return'] / highest_ret_value) * 100

    return ret_sorted

In [90]:
# Loading data into variables
stock_filter = filter_stocks(ticker_lst)
ticker_data = stock_filter[0]
ticker_lst = list(ticker_data.keys()) # Reassign original ticker list
data = pd.DataFrame()
for ticker in ticker_data:
    data[ticker] = ticker_data[ticker]

# returns = data.pct_change()
# returns.drop(index=returns.index[0], inplace = True)

data.head()

$AGN: possibly delisted; no timezone found
$AGN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no timezone found
$CELG: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no timezone found
$MON: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no timezone found
$RTN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-10-03,196.642165,177.408125,133.033023,356.980153,65.679836,161.914479,189.787306,176.124621,41.072009,6.53,...,156.255604,114.575356,37.799999,25.033068,77.623268,208.878914,699.203369,266.917586,207.141485,52.539674
2022-10-04,201.680773,182.115232,137.791853,369.675080,69.058650,169.194202,197.160871,186.548168,42.776191,6.74,...,163.120622,117.543640,42.610001,25.316833,79.079193,215.818010,709.592803,273.741807,214.508121,53.879396
2022-10-05,202.094839,183.833899,138.192860,369.715503,68.511108,168.998587,195.683403,184.592013,42.168501,6.67,...,166.479805,117.041306,41.959999,25.104012,78.620865,219.281055,714.882546,267.397382,213.288802,53.335926
2022-10-06,200.755869,179.934807,136.949683,363.152373,67.789931,168.090376,192.986126,184.717761,41.560805,6.64,...,166.999684,113.762512,41.320000,24.421209,75.751686,217.634452,704.113223,264.185237,210.329496,51.920378
2022-10-07,193.384372,177.972479,136.067430,349.999354,66.387661,160.070097,188.404757,181.350360,40.622848,6.29,...,161.174424,110.520248,37.349998,24.376871,74.671204,208.147066,684.744832,260.733107,202.124581,50.808152


In [91]:
# Function to get the total volume for a call or put of a given stock.
# ticker: yfinance Ticker class
# put: Boolean for if you want to calculate put volume. Else, put False for call volume. 
def get_options_vol(ticker, put):
    exps = ticker.options # Expiration dates of available options
    optdata = pd.DataFrame() # Data storage
    for exp in exps:
        chain = pd.DataFrame()
        if put: chain = ticker.option_chain(exp).puts['volume'] # Gets the desired columns
        else: chain = ticker.option_chain(exp).calls['volume'] # If put options are desired then use this data.
        optdata = pd.concat([optdata, chain]) # Add the calls/puts to the main dataframe. 
    return optdata.sum()['volume'] # output total volue of put/call options

# Function to calculate the PCR for each stock. 
def PCR_calc(tickers):
    pcrdata = pd.DataFrame(columns=['Ticker', 'Put Volume', 'Call Volume', 'PCR'])
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try: 
            # Get the volume for Put and Call options:
            call_options = get_options_vol(stock, False)
            put_options = get_options_vol(stock, True)
            # Calculate PCR Ratio:
            pcr = call_options / put_options # Order reversed from the formula for sake of ranking
            #print(f"Ticker: {ticker}, PCR: {pcr}")  # Debugging
            pcrdata.loc[len(pcrdata)] = [ticker, put_options, call_options, pcr]
        except Exception as e:
            print(f"Options Data Not Found {ticker}: {e} not found")  # Debugging (output error)
            pass
    return pcrdata

In [92]:
std = calculate_std(data)
std

,Standard Deviation,Rank,Score
SHOP.TO,0.036675,0,100.000000
BB.TO,0.034706,1,94.630544
PYPL,0.023866,2,65.074545
QCOM,0.023577,3,64.284995
AMZN,0.021535,4,58.719326
USB,0.021437,5,58.452267
BA,0.020557,6,56.052245
CAT,0.017753,7,48.407070
LLY,0.016873,8,46.006087
TXN,0.016711,9,45.566070


In [93]:
ret = calculate_return(data)
ret

,Return,Rank,Score
SHOP.TO,0.003137,0,100.000000
LLY,0.001928,1,61.474897
CAT,0.001796,2,57.254931
AXP,0.001552,3,49.491712
BK,0.001506,4,48.000418
QCOM,0.001437,5,45.799410
BLK,0.001431,6,45.617507
AMZN,0.001385,7,44.146281
AAPL,0.001331,8,42.419378
C,0.001076,9,34.319300


In [94]:
# Load the PCR values for each of the valid stocks into a variable
options_data = PCR_calc(ticker_lst)
options_data = options_data.sort_values(by='PCR', ascending=False)
options_data['Rank'] = [i for i in range(len(options_data))]
highest_pcr = options_data['PCR'].iloc[0]
options_data['Score'] = (options_data['PCR'] / highest_pcr) * 100
options_data.set_index('Ticker', inplace=True)


# Display the table of rankings based off PCR. 
# The rankings are based off the stocks with the greatest sentiment for if they will go up or not
# The tickers at the top of the list have a high call rate (meaning the price will go up)
pcr = options_data
pcr

Options Data Not Found BB.TO: 'volume' not found
Options Data Not Found RY.TO: 'volume' not found
Options Data Not Found SHOP.TO: 'volume' not found
Options Data Not Found T.TO: 'volume' not found
Options Data Not Found TD.TO: 'volume' not found


,Put Volume,Call Volume,PCR,Rank,Score
Ticker,,,,,
KO,11173.0,58728.0,5.256243,0,100.000000
PM,2844.0,10426.0,3.665963,1,69.744942
PG,8260.0,26726.0,3.235593,2,61.557150
PEP,5154.0,15798.0,3.065192,3,58.315269
AMZN,261121.0,660950.0,2.531202,4,48.156110
ABT,3750.0,9479.0,2.527733,5,48.090118
MRK,9984.0,22607.0,2.264323,6,43.078736
LMT,4555.0,9932.0,2.180461,7,41.483264
BK,1664.0,2832.0,1.701923,8,32.379081


In [95]:
def calculate_scoreboard(std, pcr, ret):
    """
    Merges three DataFrames (std, pcr, ret) on their index (assumed to be ticker names),
    calculates the average of their 'Score' columns, and sorts the result by 'Average Score'.
    Handles NaN values by taking the value from 'ret' where available.
    """
    # Merge std and pcr DataFrames
    merged = std[['Score']].merge(
        pcr[['Score']], left_index=True, right_index=True, suffixes=('_std', '_pcr'), how='outer'
    )
    
    # Merge the resulting DataFrame with ret
    merged = merged.merge(
        ret[['Score']].rename(columns={'Score': 'Score_ret'}),  # Rename the Score column in ret
        left_index=True,
        right_index=True,
        how='outer'
    )

    # Fill missing values: take the value from 'ret' where other columns are NaN
    merged['Score_std'] = merged['Score_std'].fillna(merged['Score_ret'])
    merged['Score_pcr'] = merged['Score_pcr'].fillna(merged['Score_ret'])
    merged['Score_ret'] = merged['Score_ret'].fillna(merged[['Score_std', 'Score_pcr']].mean(axis=1))

    # Calculate the average score from all three columns
    merged['Average Score'] = merged[['Score_std', 'Score_pcr', 'Score_ret']].mean(axis=1)

    # Sort the DataFrame by 'Average Score' in descending order
    merged_sorted = merged.sort_values(by='Average Score', ascending=False)

    return merged_sorted



calculate_scoreboard(std, pcr, ret)

,Score_std,Score_pcr,Score_ret,Average Score
SHOP.TO,100.000000,100.000000,100.000000,100.000000
AMZN,58.719326,48.156110,44.146281,50.340572
KO,23.578009,100.000000,22.051452,48.543154
LLY,46.006087,31.323013,61.474897,46.267999
CAT,48.407070,30.504270,57.254931,45.388757
QCOM,64.284995,23.905048,45.799410,44.663151
PM,28.812864,69.744942,31.086480,43.214762
AXP,44.742705,27.957383,49.491712,40.730600
BK,39.818516,32.379081,48.000418,40.066005
PG,26.120411,61.557150,26.900692,38.192751


In [96]:
def calculate_scoreboard_2(std, pcr, ret):
    """
    Merges three DataFrames (std, pcr, ret) on their index (assumed to be ticker names),
    calculates the average of 'Score_std' and 'Score_pcr', using 'Score_ret' only when 'Score_pcr' is NaN.
    The average score is strictly based on 'Score_std' and 'Score_pcr'.
    """
    # Merge std and pcr DataFrames
    merged = std[['Score']].merge(
        pcr[['Score']], left_index=True, right_index=True, suffixes=('_std', '_pcr'), how='outer'
    )
    
    # Merge the resulting DataFrame with ret
    merged = merged.merge(
        ret[['Score']].rename(columns={'Score': 'Score_ret'}),  # Rename the Score column in ret
        left_index=True,
        right_index=True,
        how='outer'
    )

    # Use 'Score_ret' where 'Score_pcr' is NaN
    merged['Score_pcr'] = merged['Score_pcr'].fillna(merged['Score_ret'])
    merged.drop(columns=['Score_ret'], inplace=True)

    # Calculate the average score using only 'Score_std' and 'Score_pcr'
    # Exclude rows where both 'Score_std' and 'Score_pcr' are NaN
    merged['Average Score'] = merged[['Score_std', 'Score_pcr']].mean(axis=1)

    # Sort the DataFrame by 'Average Score' in descending order
    merged_sorted = merged.sort_values(by='Average Score', ascending=False)

    
    return merged_sorted




scores = calculate_scoreboard_2(std, pcr, ret)

In [97]:
# Load market data into a dataframe
s_p500 = yf.Ticker('^GSPC').history(start=start_date, end=end_date)['Close']
tsx60 = yf.Ticker('^GSPTSE').history(start=start_date, end=end_date)['Close']

SPreturns = s_p500.pct_change().dropna()
TSX60Returns = tsx60.pct_change().dropna()

avg_return = (SPreturns + TSX60Returns)/2

market_indices = pd.DataFrame({'S&P 500 PCT Returns': SPreturns, 
                               'TSX 60 PCT Returns': TSX60Returns, 
                               'Average Market Return': avg_return})
market_indices.index = market_indices.index.strftime('%Y-%m-%d')
market_indices.index = pd.to_datetime(market_indices.index)

market_indices.head()

,S&P 500 PCT Returns,TSX 60 PCT Returns,Average Market Return
Date,,,
2022-10-03,0.025884,0.023693,0.024788
2022-10-04,0.030584,0.025941,0.028262
2022-10-05,-0.002018,-0.007016,-0.004517
2022-10-06,-0.010245,-0.013314,-0.011780
2022-10-07,-0.028004,-0.020860,-0.024432


In [98]:
market_variance = market_indices['Average Market Return'].var()
print(f'Market Variance: {market_variance}')

Market Variance: 6.424001518121877e-05


In [99]:
# sharpe ratio optimization

def optimal_sharpe(tickers, risk_free_rate, investment):
    # Load market data into a dataframe
    data = pd.DataFrame()
    for ticker in tickers:
        data[ticker] = ticker_data[ticker]

    def neg_sharpe(weights):
        # determining number of shares of each stock that can be bought
        shares = []
        for i in range(len(tickers)):
            allocation = investment * weights[i]  # investment allocated to this stock
            price_per_share = data.iloc[0][tickers[i].upper()]
            
            # clculate fees
            flat_fee = 3.95
            per_share_fee = allocation/price_per_share/(1000+1/price_per_share) 
            
            # choose the smaller of the two fees
            trading_fee = min(flat_fee, per_share_fee)
            
            # calculate the number of shares after deducting the fee
            effective_investment = allocation - trading_fee
            shares.append(effective_investment / price_per_share)
        
        # forming the portfolio
        portfolio = data*shares
        portfolio['total'] = portfolio.sum(axis=1)
        portfolio['daily return'] = portfolio['total'].pct_change(1)

        # calculating sharpe ratio
        er = portfolio['daily return'].mean()
        std = portfolio['daily return'].std()
        sharpe_ratio = (er-risk_free_rate)/std
        
        sharpe_ratio = sharpe_ratio*(252**0.5) # annualizing sharpe ratio by trading days

        return -sharpe_ratio #make sharpe ratio negative for minimize function

    # constraints
    def check_sum(weights): 
        return np.sum(weights)-1 #returns 0 if weights sum up to 1
    constraints = {'type': 'eq', 'fun': check_sum}

    min_weight = 1/(2*len(tickers))
    max_weight = 0.4

    bounds = [(min_weight, max_weight)]*len(tickers)

    # initial guess
    init_guess = [1.0/len(tickers)]*len(tickers)

    results = minimize(neg_sharpe, init_guess, method="SLSQP", bounds=bounds, constraints=constraints)

    return results

# Example test case: 
# optimal = optimal_sharpe(ticker_lst, 0, 100)
# print(f'Sharpe Ratio: {optimal.fun}\nOptimal Weights: \n{optimal.x}')

In [132]:
# Calculating beta of the stock portfolio
current_best = None
for i in range(min_stocks, max_stocks+1):
    current_stocks = list(scores.head(i).index)
    stock_weight_data = optimal_sharpe(current_stocks, 0, amount)
    weights = stock_weight_data.x
    sharpe_ratio = -stock_weight_data.fun
    portfolio_beta = 0
    for index, value in enumerate(weights):
        temp = pd.DataFrame()
        temp['Market'] = market_indices['Average Market Return']

        istock_return = data[current_stocks[index]].pct_change(fill_method=None).dropna()
        temp['Stock'] = istock_return
        covariance = temp.cov()
        # Add Weight adjusted beta
        portfolio_beta += value * (covariance.loc['Stock', 'Market'] / market_variance)
    print(f'Sharpe Ratio: {sharpe_ratio}\nPortfolio Beta: {portfolio_beta}\nNumber of Stocks: {i}\n')
    if current_best is None or (sharpe_ratio > current_best[0] and portfolio_beta > current_best[1]):
        current_best = (sharpe_ratio, portfolio_beta, current_stocks, weights)
    
print(f'''
      Best Sharpe Ratio: {current_best[0]}
      Best Portfolio Beta: {current_best[1]}
      Number of Stocks: {len(current_best[2])}
''')

Sharpe Ratio: 2.048883086944234
Portfolio Beta: 1.1520688765483307
Number of Stocks: 12

Sharpe Ratio: 2.0643707056337917
Portfolio Beta: 1.1173448254502438
Number of Stocks: 13

Sharpe Ratio: 2.0841794644247247
Portfolio Beta: 0.9872775080455017
Number of Stocks: 14

Sharpe Ratio: 2.0943612004140415
Portfolio Beta: 1.0013825357712962
Number of Stocks: 15

Sharpe Ratio: 2.103706090951551
Portfolio Beta: 1.0104492248672658
Number of Stocks: 16

Sharpe Ratio: 2.1119832240232785
Portfolio Beta: 0.9887169831147475
Number of Stocks: 17

Sharpe Ratio: 2.1191701167166324
Portfolio Beta: 0.9934037732896419
Number of Stocks: 18

Sharpe Ratio: 2.124899431133725
Portfolio Beta: 1.0010254693693064
Number of Stocks: 19

Sharpe Ratio: 2.1312000936562345
Portfolio Beta: 1.0045339856931976
Number of Stocks: 20

Sharpe Ratio: 2.1357171036796143
Portfolio Beta: 1.0161683426049615
Number of Stocks: 21

Sharpe Ratio: 2.1411371864567257
Portfolio Beta: 1.0200513714328825
Number of Stocks: 22

Sharpe Ratio:

In [119]:
# Store date for November, 22, 2024 in a variable:
date1, date2 = '2024-11-21', '2024-11-22'
final_portfolio = current_best[2]
final_weights = current_best[3]
Portfolio_Final.Ticker = final_portfolio
Portfolio_Final.index = Portfolio_Final.index + 1
currency, price  = [], []
for ticker in final_portfolio:
    # Get the price of the stock on the specified date
    stock = yf.Ticker(ticker).history(start=date1, end=date2)['Close'].iloc[0]
    price.append(stock)
    if ticker in stock_filter[2]:
        currency.append('USD')
    else: currency.append('CAD')
Portfolio_Final.Currency = currency
Portfolio_Final.Price = price


In [120]:
# Code to output final dataframe to a CSV file called Stocks_Group_XX.csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv(f'Stocks_Group_{group}.csv', index=False)

In [121]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight
5,SHOP.TO,148.809998,CAD,NaN,NaN,NaN
6,KO,63.759998,USD,NaN,NaN,NaN
7,AMZN,198.380005,USD,NaN,NaN,NaN
8,PM,131.210007,USD,NaN,NaN,NaN
9,PYPL,84.820000,USD,NaN,NaN,NaN
10,BB.TO,3.260000,CAD,NaN,NaN,NaN
11,QCOM,155.460007,USD,NaN,NaN,NaN
12,PG,172.750000,USD,NaN,NaN,NaN
13,PEP,160.339996,USD,NaN,NaN,NaN
14,ABT,117.260002,USD,NaN,NaN,NaN


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

---
<p style="color: #004dd3">
Akram Jamil
</p>

<p style="color: #2C8CA9">
Jester Yang
</p>

<p style="color: #3cc19d;">
Annie Wong
</p>

---